# SK: Agents using Planner

## Outline:

* Using built in SK Skills: HttpSkill, TimeSkill and MathSkill
* Defining your own tools

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

In [2]:
import semantic_kernel as sk
import os
import logging
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('__name__')
kernel=sk.Kernel(log=logger)

api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
)

## Built-in SK tools

In [3]:
kernel.import_skill(sk.core_skills.MathSkill(), "MathSkill")
kernel.import_skill(sk.core_skills.TimeSkill(), "TimeSkill")
kernel.import_skill(sk.core_skills.HttpSkill(), "HttpSkill")

{'deleteAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f65aceeadf0>,
 'getAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f65aceeaee0>,
 'postAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f65aceee070>,
 'putAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f65aceee1c0>}

In [4]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [5]:
ask = "What is the sum of 2319 and 86988?"

In [6]:
original_plan = await planner.create_plan_async(ask, kernel)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=2500 request_id=42c3c770e1c81a20dfc866d2109ed455 response_code=200


In [7]:
print(original_plan.generated_plan)

{
    "input": "2319 and 86988",
    "subtasks": [
        {"function": "MathSkill.Add", "args": {"input": 2319, "Amount": 86988}}
    ]
}


In [8]:
results = await planner.execute_plan_async(original_plan, kernel)
print(results)

89307


In [9]:
ask="whats the date today?"
plan2 = await planner.create_plan_async(ask, kernel)
print(plan2.generated_plan)
result2= await planner.execute_plan_async(plan2, kernel)
print(result2)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1537 request_id=bb13a18a75fa6ad181a072d5f6b69064 response_code=200


{
    "input": "whats the date today?",
    "subtasks": [
        {"function": "TimeSkill.today"}
    ]
}
Monday, 19 June, 2023


In [10]:
# Plans generated are not always accurate. Here is an example
ask = "What book did Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU) write?"

# IMPORTANT: Copy the skills/WriterSkill directory from https://github.com/microsoft/semantic-kerne into current directory
skills_directory = "skills"
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")

plan3 = await planner.create_plan_async(ask, kernel)
print(plan3.generated_plan)
 

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=5324 request_id=56b02f87dda447808e6119192b92412d response_code=200


{
    "input": "Tom M. Mitchell and his book",
    "subtasks": [
        {"function": "HttpSkill.getAsync", "args": {"input": "https://en.wikipedia.org/api/rest_v1/page/html/Tom_M._Mitchell"}},
        {"function": "WriterSkill.Rewrite", "args": {"style": "compact", "input": "The book Tom M. Mitchell wrote"}},
        {"function": "WriterSkill.EmailTo", "args": {"to": "recipient", "input": "The book Tom M. Mitchell wrote", "sender": "sender"}}
    ]
}


In [11]:
result3= await planner.execute_plan_async(plan3, kernel)
print(result3)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=438 request_id=405e12ba8c9bee3e6ed621ea6b6aaff7 response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1805 request_id=0c17ddb5f71e3261e8a44c19a6062392 response_code=200


Dear Recipient,

I wanted to bring to your attention a book written by Tom M. Mitchell. It's a great read and I highly recommend it.

Thank you,
Sender


## Define your own tool

In [ ]:
# TODO